In [1]:
from models.blipv2_bert import blip_pretrain

In [2]:
import yaml

config = yaml.load(open("configs/pretrain_blip.yaml", "r"), Loader=yaml.Loader)
# config = yaml.load(open("configs/finetune.yaml", "r"), Loader=yaml.Loader)

In [3]:
device = "cuda"

model = blip_pretrain(config=config)
model.to(device)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['visual.proj', 'visual.ln_post.weight', 'visual.ln_post.bias', 'visual.transformer.resblocks.23.attn.in_proj_weight', 'visual.transformer.resblocks.23.attn.in_proj_bias', 'visual.transformer.resblocks.23.attn.out_proj.weight', 'visual.transformer.resblocks.23.attn.out_proj.bias', 'visual.transformer.resblocks.23.ln_1.weight', 'visual.transformer.resblocks.23.ln_1.bias', 'visual.transformer.resblocks.23.mlp.c_fc.weight', 'visual.transformer.resblocks.23.mlp.c_fc.bias', 'visual.transformer.resblocks.23.mlp.c_proj.weight', 'visual.transformer.resblocks.23.mlp.c_proj.bias', 'visual.transformer.resblocks.23.ln_2.weight', 'visual.transformer.resblocks.23.ln_2.bias'])
freeze class_embedding
freeze positional_embedding
freeze conv1.weight
freeze ln_pre.weight
freeze ln_pre.bias
freeze transformer.resblocks.0.attn.in_proj_weight
freeze transformer.resblocks.0.attn.in_proj_bias
freeze transformer.resblocks.0.attn.out_proj.weight
freeze transfor

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

copy from bert.encoder.layer.0.intermediate.dense.weight to bert.encoder.layer.0.intermediate_query.dense.weight
copy from bert.encoder.layer.0.intermediate.dense.bias to bert.encoder.layer.0.intermediate_query.dense.bias
copy from bert.encoder.layer.0.output.dense.weight to bert.encoder.layer.0.output_query.dense.weight
copy from bert.encoder.layer.0.output.dense.bias to bert.encoder.layer.0.output_query.dense.bias
copy from bert.encoder.layer.0.output.LayerNorm.weight to bert.encoder.layer.0.output_query.LayerNorm.weight
copy from bert.encoder.layer.0.output.LayerNorm.bias to bert.encoder.layer.0.output_query.LayerNorm.bias
copy from bert.encoder.layer.1.intermediate.dense.weight to bert.encoder.layer.1.intermediate_query.dense.weight
copy from bert.encoder.layer.1.intermediate.dense.bias to bert.encoder.layer.1.intermediate_query.dense.bias
copy from bert.encoder.layer.1.output.dense.weight to bert.encoder.layer.1.output_query.dense.weight
copy from bert.encoder.layer.1.output.dense

BLIP_Pretrain(
  (visual_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): Identity()
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLine

In [4]:
import torch
state_dict = torch.load("pretrain_blipv2/checkpoint_14.pth", map_location="cpu")["model"]

In [5]:
msg = model.load_state_dict(state_dict)

In [6]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

# from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from constant import OPENAI_DATASET_MEAN, OPENAI_DATASET_STD
transform_test = transforms.Compose([
    transforms.Resize(config['image_size'],interpolation=InterpolationMode.BICUBIC),
    transforms.CenterCrop(config['image_size']),
    transforms.ToTensor(),
    transforms.Normalize(OPENAI_DATASET_MEAN, OPENAI_DATASET_STD),
    ])       

In [7]:
from PIL import Image

image = Image.open("../data/alvan-nee/alvan-nee-9M0tSjb-cpA-unsplash.jpeg")
image = transform_test(image).unsqueeze(0).to(device)


In [8]:
model.to(device)

BLIP_Pretrain(
  (visual_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): Identity()
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLine

In [9]:
model.generate(image=image)

['a small brown and white puppy sitting on an orange background']